# LLM 

autenticazione LLM: effettuare l'autenticazione all'LLM tramite chiamata http direttamente dal jupyter notebook.

NOTA: CI SONO DUE POSSIBILITA' PER CAPIRE COME FARLO
- RICERCA SU INTERNET
- al momento dell'autenticazione da browser, segnarsi tutti gli endpoint che vengono chiamati e la forma della payload che gli viene inviata.
	poi, replicare le chiamate da codice utilizzando la libreria 'requests' o 'aiohttp'

In [1]:
import requests


BASE_URL = "http://localhost:3000"
url = f"{BASE_URL}/api/v1/auths/signin"
payload = {
    "email": "eleonora.fabris@dbagroup.it", 
    "password": "Scoiattolo1!"
}
response = requests.post(url, json=payload)
token = response.json()["token"]


In [2]:
url = f"{BASE_URL}/api/v1/models"
JWT_TOKEN = token

HEADERS = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=HEADERS)

if response.status_code == 200:
    print("Autenticazione riuscita!")
    print(response.json()) # lista modelli disponibili
else:
    print(f"Errore nell'autenticazione: {response.status_code}")



Autenticazione riuscita!
{'data': [{'id': 'gemma3:4b', 'name': 'gemma3:4b', 'object': 'model', 'created': 1766489510, 'owned_by': 'ollama', 'ollama': {'name': 'gemma3:4b', 'model': 'gemma3:4b', 'modified_at': '2025-12-22T16:59:15.153912+01:00', 'size': 3338801804, 'digest': 'a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], 'parameter_size': '4.3B', 'quantization_level': 'Q4_K_M'}, 'connection_type': 'local', 'urls': [0]}, 'connection_type': 'local', 'tags': [], 'actions': [], 'filters': []}, {'id': 'falcon3:7b', 'name': 'falcon3:7b', 'object': 'model', 'created': 1766489510, 'owned_by': 'ollama', 'ollama': {'name': 'falcon3:7b', 'model': 'falcon3:7b', 'modified_at': '2025-12-22T16:51:27.8742221+01:00', 'size': 4569738977, 'digest': '472ea1c89f64f932368963bfb8c7f54bd797f2161932cf0a72766a587b2e586d', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families

In [3]:
url_new_chat = f"{BASE_URL}/api/v1/chats/new"
payload_chat = {
  "chat": {
    "name": "Lettura della bolletta",
    "system_prompt": "Rispondi solo in JSON"
  }
}
response = requests.post(url_new_chat, json=payload_chat, headers=HEADERS)
if response.status_code in [200, 201]:
    chat_data = response.json()
    chat_id = chat_data.get("id")
    print("Nuova chat creata:", chat_id)
else:
    print("Errore nella creazione chat:", response.status_code, response.text)


Nuova chat creata: da41e0b5-6bbd-410a-81a4-0fccece48205


In [4]:
file_path = "C:\DEV\LAB\electrical-energy-billing\Fattura_0000005222062506.pdf"
url_upload_file = f"{BASE_URL}/api/v1/files/?process=true"

with open(file_path, "rb") as f:
    files = {"file": f} 
    headers_file = {"Authorization": f"Bearer {JWT_TOKEN}"} 
    response = requests.post(url_upload_file, files=files, headers=headers_file)
    file_data = response.json()
    file_id = file_data.get("id")
if response.status_code in [200, 201]:
    print("File caricato correttamente!")
    print(file_data) 
else:
    print(f"Errore upload file: {response.status_code} - {response.text}")

File caricato correttamente!
{'id': 'f3aa0244-7e56-4e0e-8293-767aa2e9c7cf', 'user_id': 'd4bfe151-cbbc-420e-9550-14695802de2d', 'hash': None, 'filename': 'Fattura_0000005222062506.pdf', 'data': {'status': 'pending'}, 'meta': {'name': 'Fattura_0000005222062506.pdf', 'content_type': None, 'size': 606838, 'data': {}}, 'created_at': 1766489520, 'updated_at': 1766489520, 'status': True, 'path': '/app/backend/data/uploads/f3aa0244-7e56-4e0e-8293-767aa2e9c7cf_Fattura_0000005222062506.pdf', 'access_control': None}


In [ ]:
# import uuid

# completion_id = str(uuid.uuid4())
# session_id = str(uuid.uuid4())

payload = {
    "model": "falcon3:7b", 
    "chat_id": chat_id,
    # "id": completion_id,
    # "session_id": session_id,
    "files": [{"id": file_id}], 
    "stream": False, 
    "messages": [
        {
            "role": "user",
            "content": """
        Dal PDF fornito, estrai i valori di F1, F2, F3 e restituiscili esclusivamente in formato JSON. 
        Il JSON deve avere esattamente questa struttura:
        {
            "f1_kwh": x_kwh,
            "f2_kwh": y_kwh,
            "f3_kwh": z_kwh,
            "f1_kvarh_capacitive": x1_kvarh,
            "f1_kvarh_inductive": x2_kvarh,
            "f1_kvarh_injected": x3_kvarh,
            "f2_kvarh_capacitive": y1_kvarh,
            "f2_kvarh_inductive": y2_kvarh,
            "f2_kvarh_injected": y3_kvarh,
            "f3_kvarh_capacitive": z1_kvarh,
            "f3_kvarh_inductive": z2_kvarh,
            "f3_kvarh_injected": z3_kvarh,
            "error_msg": ""
        }
        Regole di estrazione dei dati:
        * f*_kwh:
            * Prendi i valori dalla colonna "Quantità" nella sezione "Consumo fatturato" del PDF.
            * Ogni fascia (F1, F2, F3) deve avere il proprio valore.
        * f*_kvarh_capacitive, f*_kvarh_inductive, f*_kvarh_injected:
            * Estrarre i valori direttamente dalle seguenti sezioni del PDF per ciascuna fascia:
                * f*_kvarh_capacitive → Consumo rilevato energia reattiva capacitiva
                * f*_kvarh_inductive → Consumo rilevato energia reattiva induttiva
                * f*_kvarh_injected → Consumo fatturato energia reattiva immessa
            * Non eseguire somme o calcoli: usa il valore riportato nel PDF.
            * Tutti i valori devono corrispondere alla stessa fascia oraria (F1, F2, F3).
        Se uno o più dati non sono reperibili, restituisci tutti i campi null, con questo JSON:
        {
            "f1_kwh": null,
            "f2_kwh": null,
            "f3_kwh": null,
            "f1_kvarh_capacitive": null,
            "f1_kvarh_inductive": null,
            "f1_kvarh_injected": null,
            "f2_kvarh_capacitive": null,
            "f2_kvarh_inductive": null,
            "f2_kvarh_injected": null,
            "f3_kvarh_capacitive": null,
            "f3_kvarh_inductive": null,
            "f3_kvarh_injected": null,
            "error_msg": "FAILED TO READ CONSUMPTIONS FROM ALLEDGED BILL FILE"
        }
        ATTENZIONE:
        * La risposta deve contenere solo il JSON, nessun commento, spiegazione o testo aggiuntivo
        * I nomi dei campi devono essere esattamente quelli indicati sopra
        """ }
    ],
    "params": {
        "max_new_tokens": 500, 
        "temperature": 0.7, # se 0.0 risponde sempre allo stesso modo, estremamente preciso
        "top_p": 0.9 # indica la variabilità del vocabolario
    }
}


In [ ]:
url_prompt = f"{BASE_URL}/api/chat/completed" # completions

response = requests.post(url_prompt, json=payload, headers=HEADERS)

if response.status_code == 200:
    data = response.json()
    messages = data.get("messages", [])
    assistant_msgs = [m for m in messages if m["role"] == "assistant"]
    if assistant_msgs:
        last_response = assistant_msgs[-1]["content"]
        print("JSON finale dal modello:", last_response)
    else:
        print("Nessuna risposta dell'assistant trovata")
else:
    print(f"Errore completions: {response.status_code} - {response.text}")

Errore completions: 400 - {"detail":"'NoneType' object has no attribute 'get'"}


In [ ]:
url_delete_chat = f"{BASE_URL}/api/v1/chats/{chat_id}"
response = requests.delete(url_delete_chat, headers=HEADERS)
if response.status_code in [200, 204]: # cancellazione riuscita: risposta JSON opzionale, nessun contenuto restituito
    print(f"Chat {chat_id} cancellata correttamente.")
else:
    print(f"Errore nella cancellazione: {response.status_code} - {response.text}")
